In [1]:
import pandas as pd
import numpy as np

print('Версия pandas:', pd.__version__)

Версия pandas: 2.1.4


# Функции pd.pivot_table & pd.crosstab

# Вычисление процента в сводной таблице в pandas

## Вычисление процентов с помощью функции pd.pivot_table
    
## Параметры функции pd.pivot_table()

    data: объект DataFrame
    index: столбец изначального объекта DataFrame для использования в качестве меток строк
    columns: столбец изначального объекта DataFrame, значения которого будут преобразованы в columns
    values: столбцы изначального объекта DataFrame, которые будут использоваться для значений нового датафрейма
    aggfunc: функция, используемая для агрегирования, по умолчанию - 'mean'
    fill_value: на какие значения будут заменены отсутствующие значения в результирующем объекте DataFrame
    margins: Промежуточные итоги в результирующем объекте DataFrame, по умолчанию False. Если margins=True,
    в результат будут добавлены строка и колонка с меткой All и частичными групповыми агрегациями.
    margins_name: по умолчанию All, название метки строки и столбца вместо 'All, если параметр margins=True
    dropna: следует ли исключать столбцы, все записи которых являются NaN
    sort: по умолчанию sort=True - метки строк и столбцов отсортированы в лексикографическом порядке

In [2]:
df_sales=pd.read_csv('files/sales_Emily_James_Charlie')
df_sales

,salesperson,product,sales_amount
0,Emily,A,100
1,James,B,200
2,Charlie,C,300
3,Emily,A,150
4,Charlie,B,250
5,James,C,200


# Создаем две сводные таблицы с помощью pd.pivot_table()

- pt_sum - с суммой продаж, но без итогов All (margins=False)
- pt_sum_all - с суммой продаж, с итогами All (margins=True)

In [3]:
pt_sum = pd.pivot_table(df_sales,index='salesperson',columns='product', values='sales_amount', 
                                 aggfunc='sum', fill_value=0)
pt_sum

product,A,B,C
salesperson,,,
Charlie,0,250,300
Emily,250,0,0
James,0,200,200


In [4]:
# если margins=True

pt_sum_all=pd.pivot_table(df_sales,index='salesperson',columns='product', values='sales_amount', 
               aggfunc='sum', fill_value=0, margins=True)
pt_sum_all

product,A,B,C,All
salesperson,,,,
Charlie,0,250,300,550
Emily,250,0,0,250
James,0,200,200,400
All,250,450,500,1200


## Находим долю от общей суммы продаж

## Задача № 1
#### Создать сводную таблицу с процентом от общего объема продаж для каждого продавца по каждому типу товара (A, B, C)

#### 1 вариант 
    Создаем сводную таблицу с агрегирующей функцией 'sum' и итогами (margins=True), а потом применяем функцию деления
#### 2 вариант 
    Сразу создаем сводную таблицу с процентами, передав соответствующую lambda-функцию в параметр aggfunc

## Вариант 1

        Для вычислений используем сохраненную сводную таблицу (pt_sum_all), 
        из этой сводной таблицы получаем общую сумму с помощью метода iloc (pt_sum_all.iloc[-1,-1])
        Делим каждое значение сводной таблицы на общую сумму. Используем метод div. 
        Умножаем результат на 100 и округляем до сотых. Используем функцию round.

In [5]:
# сводная таблица с итогами (margins=True) и общая сумма (правый нижний угол таблицы - iloc[-1,-1])

display(pt_sum_all, pt_sum_all.iloc[-1,-1])

product,A,B,C,All
salesperson,,,,
Charlie,0,250,300,550
Emily,250,0,0,250
James,0,200,200,400
All,250,450,500,1200


1200

In [6]:
# Применяем деление. Умножаем на 100. Округляем результат

round((pt_sum_all.div(pt_sum_all.iloc[-1,-1])*100),2)

product,A,B,C,All
salesperson,,,,
Charlie,0.00,20.83,25.00,45.83
Emily,20.83,0.00,0.00,20.83
James,0.00,16.67,16.67,33.33
All,20.83,37.50,41.67,100.00


In [7]:
# если в качестве общей суммы возьмем sum(df_sales['sales_amount'])

# можно создать обычную сводную таблицу, а можно с итогами

display(round((pt_sum.div(sum(df_sales['sales_amount']))*100),2),
        round((pt_sum_all.div(sum(df_sales['sales_amount']))*100),2))

product,A,B,C
salesperson,,,
Charlie,0.00,20.83,25.00
Emily,20.83,0.00,0.00
James,0.00,16.67,16.67


product,A,B,C,All
salesperson,,,,
Charlie,0.00,20.83,25.00,45.83
Emily,20.83,0.00,0.00,20.83
James,0.00,16.67,16.67,33.33
All,20.83,37.50,41.67,100.00


## Вариант 2

### Для решения этой задачи нам нужно использовать лямбда-функцию в параметре aggfunc

    Для решения этой задачи нам нужно использовать лямбда-функцию в параметре aggfunc
    
    lambda x: round(sum(x)/sum(df_sales['sales_amount']) * 100, 2), где...

    sum(df_sales['sales_amount']) - это общая сумма всех продаж 
    sum(x) - сумма по группе (то же самое, что df_sales.groupby(['salesperson','product'])['sales_amount'].agg('sum'))

   #### aggfunc=lambda x: round(sum(x)/sum(df_sales['sales_amount']) * 100, 2)
    
    все значения по группе суммируются sum(x)
    полученная сумма группы делится на общую сумму sum(df_sales['sales_amount'])
    результат деления умножается на 100 и округляется до сотых (функция 'round')
    
#### все это происходит в момент создания сводной таблицы

In [8]:
pt_percent = pd.pivot_table(df_sales, 
                             values='sales_amount',
                             index='salesperson',
                             columns='product',
                             fill_value=0, aggfunc=lambda x: round(sum(x)/sum(df_sales['sales_amount']) * 100, 2))
# если ставим margins=True

pt_percent_2=pd.pivot_table(df_sales, 
                             values='sales_amount',
                             index='salesperson',
                             columns='product',
                             fill_value=0, aggfunc=lambda x: round(sum(x)/sum(df_sales['sales_amount']) * 100, 2), margins=True)

display(pt_percent, pt_percent_2)

product,A,B,C
salesperson,,,
Charlie,0.00,20.83,25.00
Emily,20.83,0.00,0.00
James,0.00,16.67,16.67


product,A,B,C,All
salesperson,,,,
Charlie,0.00,20.83,25.00,45.83
Emily,20.83,0.00,0.00,20.83
James,0.00,16.67,16.67,33.33
All,20.83,37.50,41.67,100.00


#### создание сводной таблицы с процентами без использования pivot_table

In [9]:
# Мы все то же самое можем сделать, используя groupby + agg + unstack 
# но так мы не получим итоговых показателей

table_percent=round(df_sales.groupby(['salesperson','product'])['sales_amount']
      .agg('sum')/sum(df_sales['sales_amount']) * 100, 2).unstack(fill_value=0)
table_percent

product,A,B,C
salesperson,,,
Charlie,0.00,20.83,25.00
Emily,20.83,0.00,0.00
James,0.00,16.67,16.67


In [10]:
pt_percent==table_percent

product,A,B,C
salesperson,,,
Charlie,True,True,True
Emily,True,True,True
James,True,True,True


## Задача № 2
#### Определить, какую долю от общей суммы продаж товара категории 'B' составляют продажи Чарли

    
    Решить задачу помогает метод apply, примененный к ранее созданной сводной таблице pt_sum без итогов (margins=False)
    по умолчанию метод apply применяется к столбцам объекта DataFrame
    В метод apply передаем lambda-функцию (lambda x: x*100/sum(x))
    x - это значение колонки объекта DataFrame (значение колонки сводной таблицы в переменной pt_sum)
    

# Находим долю по столбцу 

## - используем apply(axis=0 - по умолчанию)

In [11]:
# Находим долю по столбцу 
# Применяем lambda-функцию к столбцам заранее созданной сводной таблицы в переменной pt_sum

pt_percent_column = round(pt_sum.apply(lambda x: x*100/sum(x)),2)
display(pt_sum, pt_percent_column)

product,A,B,C
salesperson,,,
Charlie,0,250,300
Emily,250,0,0
James,0,200,200


product,A,B,C
salesperson,,,
Charlie,0.0,55.56,60.0
Emily,100.0,0.00,0.0
James,0.0,44.44,40.0


## - используем div(axis=1-по умолчанию)

     Альтернативный вариант решения - использование метода div(axis=1-по умолчанию)
     + использование в качестве делителя последней строки сводной таблицы с итогами (серия pt_sum_all.iloc[-1,:])

In [12]:
# Находим долю по столбцу 
# по умолчанию в методе div параметр axis=1 
# используем серию pt_sum_all.iloc[-1,:] - берем строку All
display(pt_sum_all,
        pt_sum_all.iloc[-1,:],
        round((pt_sum_all.div(pt_sum_all.iloc[-1,:])*100),2))

product,A,B,C,All
salesperson,,,,
Charlie,0,250,300,550
Emily,250,0,0,250
James,0,200,200,400
All,250,450,500,1200


product
A       250
B       450
C       500
All    1200
Name: All, dtype: int64

product,A,B,C,All
salesperson,,,,
Charlie,0.0,55.56,60.0,45.83
Emily,100.0,0.00,0.0,20.83
James,0.0,44.44,40.0,33.33
All,100.0,100.00,100.0,100.00



#### Задача №2 : определить, какую долю от общей суммы продаж товара категории 'B' составляют продажи Чарли 
#### Ответ: 55.56 %


## Задача № 3
#### Определить, какую долю от общей суммы продаж Джеймса составили продажи товара категории 'C'

    Решить задачу помогает метод apply с параметром axis=1, примененный к сводной таблице pt_sum без итогов (margins=False).
    Если параметр axis=1, то метод apply применяется к строкам объекта DataFrame
    В метод apply передаем ту же самую lambda-функцию (lambda x: x*100/sum(x))

# Находим долю по строке 

## - используем apply(axis=1) 

In [13]:
# Находим долю по строке 
# ставим в методе apply параметр axis=1

pt_percent_row = round(pt_sum.apply(lambda x: x*100/sum(x), axis=1),2)
display(pt_sum, pt_percent_row)

product,A,B,C
salesperson,,,
Charlie,0,250,300
Emily,250,0,0
James,0,200,200


product,A,B,C
salesperson,,,
Charlie,0.0,45.45,54.55
Emily,100.0,0.00,0.00
James,0.0,50.00,50.00


## - используем div(axis=0)

    Альтернативный вариант решения - использование метода div(axis=0 - для деления столбцов, а не строк)
    + использование в качестве делителя последнего столбца сводной таблицы с итогами (серия pt_sum_all.iloc[:,-1])

In [14]:
# Находим долю по строке
# в методе div выставляем параметр axis=0 
# используем серию pt_sum_all.iloc[:,-1] - берем столбик All

display(pt_sum_all,
        pt_sum_all.iloc[:,-1],
        round((pt_sum_all.div(pt_sum_all.iloc[:,-1], axis=0)*100),2))

product,A,B,C,All
salesperson,,,,
Charlie,0,250,300,550
Emily,250,0,0,250
James,0,200,200,400
All,250,450,500,1200


salesperson
Charlie     550
Emily       250
James       400
All        1200
Name: All, dtype: int64

product,A,B,C,All
salesperson,,,,
Charlie,0.00,45.45,54.55,100.0
Emily,100.00,0.00,0.00,100.0
James,0.00,50.00,50.00,100.0
All,20.83,37.50,41.67,100.0



#### Задача №3: определить, какую долю от общей суммы продаж Джеймса составили продажи товара категории 'C' 
#### Ответ: 50 %


## Вычисление процентов с помощью функции pd.crosstab
    
## Параметры функции pd.crosstab()

    Параметры функции pd.crosstab
    
    index - значения изначальной структуры для группировки по строкам (метки строк результирующей структуры)
    columns - значения изначальной структуры для группировки по столбцам (метки столбцов результирующей структуры)
    values - обязательно указывать одновременно с aggfunc - значения для агрегирования
    aggfunc - агрегирующая функция - по умолчанию 'count', для вычисления % меняем на 'sum'
    margins - ставим в позицию True, если хотим результат с итогами
    normalize {‘all’, ‘index’, ‘columns’} - обязательно указываем, чтобы подсчитать...
    'all' - долю каждого агрегированного значения таблицы
    'index' - долю агрегированного значения по соответствующей строке
    'columns' - долю агрегированного значения по соответвующему столбцу

In [15]:
df_sales

,salesperson,product,sales_amount
0,Emily,A,100
1,James,B,200
2,Charlie,C,300
3,Emily,A,150
4,Charlie,B,250
5,James,C,200


## Пример № 1 (доля каждого значения от общей суммы продаж) normalize='all'

In [16]:
# вычисление процента от общей суммы продаж
# normalize='all' или True
# округляем и умножаем на 100
(pd.crosstab(index=df_sales['salesperson'], 
            columns=df_sales['product'], 
            values=df_sales['sales_amount'], 
            aggfunc='sum',
            normalize='all', 
            margins=True)*100).round(2)

product,A,B,C,All
salesperson,,,,
Charlie,0.00,20.83,25.00,45.83
Emily,20.83,0.00,0.00,20.83
James,0.00,16.67,16.67,33.33
All,20.83,37.50,41.67,100.00


In [17]:
# отключив normalize, видим NaN на месте отсутствующих значений

pd.crosstab(index=df_sales['salesperson'], 
            columns=df_sales['product'], 
            values=df_sales['sales_amount'], 
            aggfunc='sum', 
            margins=True)

product,A,B,C,All
salesperson,,,,
Charlie,NaN,250.0,300.0,550
Emily,250.0,NaN,NaN,250
James,NaN,200.0,200.0,400
All,250.0,450.0,500.0,1200


In [18]:
df_sales.pivot_table(index='salesperson', 
            columns='product', 
            values='sales_amount', 
            aggfunc='sum', 
            margins=True)

product,A,B,C,All
salesperson,,,,
Charlie,NaN,250.0,300.0,550
Emily,250.0,NaN,NaN,250
James,NaN,200.0,200.0,400
All,250.0,450.0,500.0,1200


## Пример № 2 (доли по столбцу) normalize='columns'

In [19]:
# вычисление процента по столбцу
# normalize='columns'
# в последнем столбце - какова доля каждого продавца
(pd.crosstab(df_sales['salesperson'], df_sales['product'], values=df_sales['sales_amount'], aggfunc='sum',
            normalize='columns', margins=True)*100).round(2)

product,A,B,C,All
salesperson,,,,
Charlie,0.0,55.56,60.0,45.83
Emily,100.0,0.00,0.0,20.83
James,0.0,44.44,40.0,33.33


## Пример № 3 (доли по строке) normalize='index'

In [20]:
# вычисление процента по строке
# normalize='index'
# в последней строке - какова доля каждой категории товара
(pd.crosstab(df_sales['salesperson'], df_sales['product'], values=df_sales['sales_amount'], aggfunc='sum',
            normalize='index', margins=True)*100).round(2)

product,A,B,C
salesperson,,,
Charlie,0.00,45.45,54.55
Emily,100.00,0.00,0.00
James,0.00,50.00,50.00
All,20.83,37.50,41.67
